In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import gpytorch

sns.set_style("whitegrid")
sns.set_palette("bright")

sns.set(font_scale=2.0)
sns.set_style('whitegrid')

import sys
sys.path.append("../")
from volatilitygp.likelihoods import VolatilityGaussianLikelihood
from volatilitygp.models import SingleTaskVariationalGP as SingleTaskCopulaProcessModel

In [ ]:
np.random.seed(2019)
torch.random.manual_seed(2019)

In [ ]:
F0 = 10 ## init price
V0 = 0.2 ## init price
mu = 0.05 ## rate of return

alpha = 1.25
beta = 0.9
rho = -0.2

T = 1 ## Time of Simulation
steps = 400 ## steps per time
dt = 1./(steps * T) ## delta t

dW = np.random.normal(0, np.sqrt(dt), steps*T)
dZ = rho * dW + np.sqrt(1 - rho **2) * np.random.normal(0, np.sqrt(dt), steps*T)

In [ ]:
F = np.zeros(steps*T)
V = np.zeros(steps*T)

F[0] = F0
V[0] = V0

for t in range(1, steps*T):
    F[t] = F[t-1] + V[t-1] * (F[t-1])**beta * dW[t]
    V[t] = V[t-1] + alpha * V[t-1]*dZ[t]

In [ ]:
plt.plot(V)

In [ ]:
fig, ax = plt.subplots(dpi=100)
ax.plot(F, label='Price')
ax2 = ax.twinx()
ax2.plot(V, color='OrangeRed', label='Vol')

ax.set_ylabel("Price")
ax2.set_ylabel("Vol")

fig.legend()
sns.despine()

In [ ]:
# log_returns =np.log(F[1:]/F[:-1])
scaled_returns = (F[1:] - F[:-1]) / (F[:-1]**beta) / dt**0.5

In [ ]:
fig, ax = plt.subplots(dpi=100)
ax.plot(scaled_returns, label='Log Returns')
ax2 = ax.twinx()
ax2.plot(V, color='OrangeRed', label='Vol')

ax.set_ylabel("Price")
ax2.set_ylabel("Vol")

fig.legend()
sns.despine()

## Now apply GCPV

In [ ]:
full_x = torch.FloatTensor(np.arange(steps*T-1))
full_y = torch.FloatTensor(scaled_returns)

train_x = full_x[:250]
train_y = full_y[:250]

In [ ]:
likelihood = VolatilityGaussianLikelihood()
model = SingleTaskCopulaProcessModel(
    init_points=torch.linspace(0, T*steps, 100).view(-1,1), likelihood=likelihood, use_piv_chol_init=False,
    mean_module = gpytorch.means.ZeroMean(),
)


In [ ]:
# this is for running the notebook in our testing framework
import os
smoke_test = ('CI' in os.environ)
training_iterations = 2 if smoke_test else 250


# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam([
    {"params": model.parameters()}, 
    # {"params": likelihood.parameters(), "lr": 0.1}
], lr=0.1)

# "Loss" for GPs - the marginal log likelihood
# num_data refers to the number of training datapoints
mll = gpytorch.mlls.VariationalELBO(likelihood, model, train_y.numel())

In [ ]:
print_every = 50
for i in range(training_iterations):
    # Zero backpropped gradients from previous iteration
    optimizer.zero_grad()
    # Get predictive output
    with gpytorch.settings.num_gauss_hermite_locs(75):
        output = model(train_x)
        # Calc loss and backprop gradients
        loss = -mll(output, train_y)
        loss.backward()
        if i % print_every == 0:
            print('Iter %d/%d - Loss: %.3f' % (i + 1, training_iterations, loss.item()))
        optimizer.step()

In [ ]:
model.eval();
likelihood.eval();
predictive = model(train_x)
pred_scale = likelihood(predictive, return_gaussian=False).scale.mean(0).detach()

In [ ]:
plt.plot(predictive.mean.detach())

In [ ]:
dt**0.5

In [ ]:
fig, ax = plt.subplots(dpi=100)
plt.plot(train_x, pred_scale, label = "Predicted")
plt.plot(full_x, V[1:], label = "Actual", color = "orangered")
plt.xlabel("x")
plt.ylabel("sigma(x)")
plt.legend()

# fig, ax = plt.subplots(dpi=100)
# ax.plot(train_x, pred_scale, label = "Predicted")
# ax2 = ax.twinx()
# ax2.plot(train_x, V[1:], label = "Actual", color='orangered')

# ax.set_ylabel("Predicted Vol")
# ax2.set_ylabel("Actual Vol")


# fig.legend()
sns.despine()

In [ ]:
plt.plot(predictive.mean.detach())
plt.title("Posterior Mean")

In [ ]:
%pdb

In [ ]:
fantasy_model = model.get_fantasy_model(full_x[250:].view(-1,1), full_y[250:], targets_are_gaussian=False)

In [ ]:
with gpytorch.settings.cholesky_jitter(1e-3):
    fant_dist = fantasy_model.posterior(full_x).mvn
    predictive_dist = likelihood(fant_dist, return_gaussian = False)

In [ ]:

predictive_scale = predictive_dist.scale
predictive_scale_mean = predictive_scale.mean(0).detach()
predictive_scale_std = predictive_scale.std(0).detach()

In [ ]:
predictive_scale.shape

In [ ]:
plt.plot(fantasy_model(full_x).mean.detach())# - model(full_x).mean.detach() + fantasy_model.mean_module.constant.detach())
plt.plot(model(full_x).mean.detach())


In [ ]:
orig_scale = likelihood(model(full_x) , return_gaussian=False).scale

orig_scale_mean = orig_scale.mean(0).detach()
orig_scale_std = orig_scale.std(0).detach()

In [ ]:
palette = sns.light_palette("#57068c", 10, reverse=True)
palette

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (8, 6.5))

ax2 = ax.twinx()

# ax.scatter(train_x, train_y, color = palette[4], label = "Training Points", marker = "x", s = 100, alpha = 0.3)
# ax.scatter(test_points, test_values, color = palette[2], label = "Fantasy Points", s = 100, alpha = 0.3)

ax2.plot(full_x, orig_scale_mean.detach(), label = "Original Model", color = palette[-2], linewidth=3)
ax2.fill_between(full_x, orig_scale_mean - 2 * orig_scale_std,
                orig_scale_mean + 2 * orig_scale_std, alpha = 0.1, color = palette[-2])

ax2.plot(full_x, predictive_scale_mean.detach(), label = "Fantasy Prediction", color = palette[0], linewidth=3)
ax2.fill_between(full_x, predictive_scale_mean - 2*predictive_scale_std,
                predictive_scale_mean + 2* predictive_scale_std, alpha = 0.1, color = palette[0])

ax.scatter(full_x[:250], train_y, label = "Training Points", color = palette[4], marker = "x", s = 100, alpha = 0.3)
ax.scatter(full_x[250:], full_y[250:], label = "Fantasy Points", color = palette[2], s = 100, alpha = 0.3)

# plt.plot(full_x[:250], V[1:][:250], label = "True Volatility", color = palette[4], linewidth=3)
# plt.plot(full_x[250:], V[1:][250:], label = "Fantasy Points", color = palette[2], linewidth=3)
plt.plot(full_x, V[1:], linestyle="--", color = palette[4], linewidth=3, zorder=0)
ax2.set_ylim((0, 0.6))
ax.set_ylim((-0.6, 0.6))
ax.set_ylabel("y")
# plt.legend()
# plt.ylim((0, 1))
plt.xlim((-5, 405))
ax.grid()
# plt.grid()
ax.set_xlabel("x")
plt.ylabel("Volatility")
# plt.savefig("fantasization_gpcv.pdf", bbox_inches="tight")

In [ ]:
fig, ax = plt.subplots(2, 1, figsize = (8, 7), sharex=True, sharey=True, dpi=300)

ax2 = ax[0].twinx()
ax3 = ax[1].twinx()

ax2.scatter(full_x[:250], train_y, label = "Training Points", 
            color = "#d71e5e", marker = "x", s = 100, zorder=0, alpha = 0.5)
ax3.scatter(full_x[:250], train_y, label = "Training Points", 
            color = "#d71e5e", marker = "x", s = 100, zorder=0, alpha = 0.1)
ax3.scatter(full_x[250:], full_y[250:], label = "Fantasy Points", 
            color = "#d71e5e", s = 100, marker = "x", zorder=0, alpha = 0.5)

ax[0].plot(full_x, orig_scale_mean.detach(), label = "Original Model", color = palette[3], linewidth=3,
          zorder=100)
ax[0].fill_between(full_x, orig_scale_mean - 2 * orig_scale_std,
                orig_scale_mean + 2 * orig_scale_std, alpha = 0.2, color = palette[3],
                  zorder=100)

ax[1].plot(full_x, predictive_scale_mean.detach(), label = "Fantasy Prediction", color = palette[0], linewidth=3,
          zorder=100)
ax[1].fill_between(full_x, predictive_scale_mean - 2*predictive_scale_std,
                predictive_scale_mean + 2* predictive_scale_std, alpha = 0.2, color = palette[0],
                  zorder=100)


ax2.grid()
ax3.grid()

ax[0].plot(full_x, V[1:], linestyle="--", color = "#6d6d6d", linewidth=3, zorder=50)
ax[1].plot(full_x, V[1:], linestyle="--", color = "#6d6d6d", linewidth=3, zorder=50)

# ax2.set_ylim((0, 0.6))
ax[0].set_ylabel("Volatility")
ax[1].set_ylabel("Volatility")
ax[0].set_ylim((0., 0.35))

ax2.set_ylabel("y")
ax3.set_ylabel("y")
# ax2.set_ylim((-1.5, 0.5))
# ax3.set_ylim((-1.5, 0.5))
ax[0].set_xlabel("x")
ax[1].set_xlabel("x")
# plt.savefig("fantasization_svgp_gpcv.pdf", bbox_inches = "tight")